In [10]:
with open("for_gemini.txt", "r", encoding='utf-8') as file: 
    ad_text = file.read() 
text = ad_text

In [2]:
len(text)

55092

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/e5-small")
# text = "Din jobbannons eller annan text här"
embedding = model.encode(text)  # numpy-array


c:\Users\Lukas\VSCode\python\AI_2_group_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Lukas\VSCode\python\AI_2_group_project\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lukas\.cache\huggingface\hub\models--intfloat--e5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administ

In [13]:
import plotly.express as px
px.scatter(embedding)

In [19]:
embedding #.tofile("embedding.txt")
import numpy as np
np.savetxt("embedding.txt", embedding)

# EMBEDDING
1. Initiera embedding modell
2. Ha jobbanonnser redo
3. Embedda alla texter 
4. Ha en query för att fråga gemini
5. Beräkna likheter med cosine
6. Skicka in till gemini

In [20]:
# 1. Initiera embedding modell
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("intfloat/e5-small")


In [21]:
# 2. Ha jobbanonnser redo
with open("for_gemini.txt", "r", encoding='utf-8') as file: 
    ad_text = file.read() 
text = ad_text

In [67]:
# doc_embeddings = [embedder.encode(doc) for doc in documents]

# 3. Embedda alla texter/ text i detta exempel
doc_embedding = embedder.encode(text)
doc_embedding[:10]


array([-0.01374749, -0.01689037, -0.0667139 , -0.0358971 ,  0.02319343,
       -0.0735288 ,  0.00993745,  0.05012561, -0.05575731,  0.04345234],
      dtype=float32)

In [ ]:
# 4. Ha en query med en ord som den ska hitta
# query = f"""Du är en rekryterare inom Media, Kultur, Design.
#     Plocka ut 5 av vanligaste kraven och 5 vanligaste meriterade och antal av dessa som efterfrågas i dessa jobbannonser, samt en sammanfattning av krav och meriter som du skriver ut:

#     Output ska vara i detta formatet enbart: 

#     {{
#         "sammanfattning": ["sammanfattning"]
#         "krav": [erfarenhet1, erfarenhet2, ...]
#         "antal krav": [summa erfarenhet1..]
#         "meriterande": [meriterande1,meriterande2, ... ]
#         "antal meriterande": [summa meriterande1..]
        
#     }}
#     """
query = "jag söker jobb som journalist"
query = "pilot"
query_embedding = embedder.encode(query) #embedda queryn för snabbhet
# query_embedding[:10]

In [ ]:
from scipy.spatial.distance import cosine
# 5. Beräkna likheter med cosine
similaritys = 1 - cosine(query_embedding, doc_embedding) # om du har flera texter: emb) for emb in doc_embedding]
print(f"Likhetspoäng: {similaritys}") # Vad betyder siffran? Likhetspoäng mellan de 2 vektorerna (query och text). ligger mellan 0 och 1, 1 är maximal likhet, 0 ingen likhet
if similaritys > 0.8: # Tröskel - Eftersom texten innehåller mycket så verkar detta vara gränsen, ev. 0.81 isåfall (innehåller 55.000 ord/600 rader)
    print("Hög likhet")
else: 
    print("Låg likhet")

Likhetspoäng: 0.7502351999282837
Låg likhet


### Utifrån detta kan man: 
1. Hitta ord eller korta meningar som matchar hela långa texten (Typ klart redan?)
2. Ranka annonser (måste skickas in en och en isåfall!) och sedan använda den annons som har högst poäng till något kul.    
     - då kan man använda den annonsen som har högst betyg och be gemini göra något med den, tex sammanfatta, räkna något osv
3. ....


### 2. Ranka annonser

In [173]:

# with open("for_gemini.txt", "r", encoding='utf-8') as file: 
#     ad_text = file.read() 
import duckdb 
with duckdb.connect("../../ads_data.duckdb") as con: 
    df = con.execute("SELECT * FROM mart.mart_kultur_media").df()
grouped_df = df.groupby("occupation")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()
grouped_df.head(30)



,occupation,vacancies,description
0,Tolk,566,Vi söker utbildare inom japanska. \nUppdraget ...
1,Bibliotekarie,93,I Knivsta går framtidsandan att ta på. Läget m...
2,Technical Operation Manager/TOM,91,The Opportunity:\nWith a pioneering technology...
3,Layoutare,84,Vi söker medarbetare till vårt huvudkontor i M...
4,Skådespelare,54,Teater Scarlet har spelat Mordgåtor sedan 2017...
5,Skolbibliotekarie,47,Vad vi gör och vill\nTyresö gymnasium är inne ...
6,Fotograf,45,Vi söker dig som vill utvecklas som bostadsfot...
7,Butikskommunikatör/Visual merchandiser,39,WHAT YOU’LL DO​ \nAs a Global visual & commerc...
8,Översättare,36,______________________________________________...
9,"Projektledare: kultur, media, film, dataspel",34,Om myndigheten\nForum för levande historia är ...


In [154]:
df_journalist = df[df["occupation"] == "Bibliotekarie"]
len(df_journalist["description"]) #18 annonser

57

In [ ]:
query = "jag söker jobb som bibliotekarie" # tar 27 sekunder att söka på 57 annonser
query_embedding = embedder.encode(query) #embedda queryn för snabbhet
# 5. Beräkna likheter med cosine
value_list = []
for data in df_journalist["description"]:
    doc_embedding = embedder.encode(data) #embedda varje description
    similaritys = 1 - cosine(query_embedding, doc_embedding) # räkna ut likhetspoäng utifrån query
    # if similaritys > 0.8:
    value_list.append(similaritys) # Lägg in i lista
value_list

[np.float32(0.8151097),
 np.float32(0.8418137),
 np.float32(0.82768875),
 np.float32(0.83960634),
 np.float32(0.85021055),
 np.float32(0.85311973),
 np.float32(0.8504383),
 np.float32(0.8467103),
 np.float32(0.8614044),
 np.float32(0.83192587),
 np.float32(0.840357),
 np.float32(0.8448089),
 np.float32(0.84067047),
 np.float32(0.8193293),
 np.float32(0.8203386),
 np.float32(0.8239514),
 np.float32(0.852408),
 np.float32(0.8490208),
 np.float32(0.83127666),
 np.float32(0.82288945),
 np.float32(0.83781165),
 np.float32(0.8479891),
 np.float32(0.8381494),
 np.float32(0.8270408),
 np.float32(0.84799),
 np.float32(0.81559134),
 np.float32(0.80938965),
 np.float32(0.82496876),
 np.float32(0.82550365),
 np.float32(0.8132723),
 np.float32(0.83428556),
 np.float32(0.82527065),
 np.float32(0.8422712),
 np.float32(0.83924776),
 np.float32(0.8521172),
 np.float32(0.82768875),
 np.float32(0.83960634),
 np.float32(0.85021055),
 np.float32(0.85311973),
 np.float32(0.8467103),
 np.float32(0.8614044),


In [172]:
# Bästa indexet från listan
best_index = np.argmax(value_list)
best_index

np.int64(8)

In [ ]:
# Bästa jobbanonnsen som matchar query
best_text = df_journalist.iloc[best_index].reset_index()
best_text.iloc

'Lunds universitet grundades 1666 och rankas återkommande som ett av världens främsta lärosäten. Här finns omkring 47 000 studenter och mer än 8 800 medarbetare i Lund, Helsingborg och Malmö. Vi förenas i vår strävan att förstå, förklara och förbättra vår värld och människors villkor.\n\n\nLunds universitet välkomnar sökande med olika bakgrund och erfarenheter. Jämställdhet, lika villkor och mångfald är grundläggande principer för alla delar av vår verksamhet.\n\n\n\r\n\r\nBeskrivning av arbetsplatsen\nHT-biblioteken vid fakulteten för Humaniora och teologi har omkring 30 medarbetare som arbetar vid tre fysiska enheter: Asien-, LUX- och SOL-biblioteket.\r\n\r\nDessa är belägna nära varandra och fungerar som en verksamhet. HT-biblioteken har en öppen och välkomnande arbetsmiljö där vi alla tillsammans bidrar till verksamhetens bästa.\n\n\xa0\n\nVi erbjuder\nLunds universitet är en statlig myndighet vilket innebär att du får särskilda förmåner, generös semester och en förmånlig tjänstepe

In [ ]:
# 6. Skicka in till gemini

In [ ]:
# 7. Skriv ut svar